In [2]:
import numpy as np
import cv2
import os

def hangulFilePathImageRead ( filePath ) :

    stream = open( filePath.encode("utf-8") , "rb")
    bytes = bytearray(stream.read())
    numpyArray = np.asarray(bytes, dtype=np.uint8)

    return cv2.imdecode(numpyArray , cv2.IMREAD_UNCHANGED)

# 이미지 전처리

In [12]:
directory_path = r'../img_data/raw_data'
output_path = f'../img_data/processed_data'

def img_crop(image, x_val, y_val, w_val, h_val):
    x = x_val; y = y_val; 
    w = w_val; h = h_val; 
    crop_img = image[y:y+h, x:x+w]
    return crop_img

cnt = 0
file_name_list = os.listdir(directory_path)

for file_name in file_name_list:

    directory_path_2 = f'{directory_path}/{file_name}'
    file_list = os.listdir(directory_path_2)

    if file_name in ['down', 'left', 'right', 'up']:
        x_val = 154; y_val = 170; 
        w_val = 340; h_val = 250; 
    elif file_name in ['1', '2', '3', '4', '5']:
        x_val = 500; y_val = 170; 
        w_val = 175; h_val = 250; 

    for name in file_list:
        
        cnt += 1
        file_path = f'{directory_path_2}/{name}'
        img_ori = hangulFilePathImageRead(file_path)

        # # 기울어진 이미지 수평보정 및 크롭
        gray = cv2.cvtColor(img_ori, cv2.COLOR_BGR2GRAY)
        blurred = cv2.GaussianBlur(gray, (5, 5), 0)  # 가우시안 블러 적용
    
        # 이진화, 외각선 검출
        _, binary = cv2.threshold(blurred, 0, 255, cv2.THRESH_BINARY + cv2.THRESH_OTSU)
        contours, _ = cv2.findContours(binary, cv2.RETR_LIST, cv2.CHAIN_APPROX_SIMPLE)

        # 외각선을 둘레 길이를 기준으로 정렬한 후 리스트로 변환합니다.
        contours = sorted(contours, key=lambda x: cv2.arcLength(x, True), reverse=True)
        
        # 배경을 제외한 가장큰(교구)외각선 선택
        largest_contour = contours[1] 
        # 교구의 꼭지점 찾기
        epsilon = 0.04 * cv2.arcLength(largest_contour, True)
        approx = cv2.approxPolyDP(largest_contour, epsilon, True)

        image_with_contours = blurred.copy()

        # 꼭지점 좌표 위치에 맞춰 저장
        edge_point = []

        for point in approx:
            x, y = point[0]
            edge_point.append((x, y))

        left_top_screen = (float('inf'), float('inf'))
        right_top_screen = (float('-inf'), float('inf'))
        left_bottom_screen = (float('inf'), float('-inf'))
        right_bottom_screen = (float('-inf'), float('-inf'))

        for x, y in edge_point:
            if x + y < left_top_screen[0] + left_top_screen[1]:
                left_top_screen = (x, y)
            if x - y > right_top_screen[0] - right_top_screen[1]:
                right_top_screen = (x, y)
            if x - y < left_bottom_screen[0] - left_bottom_screen[1]:
                left_bottom_screen = (x, y)
            if x + y > right_bottom_screen[0] + right_bottom_screen[1]:
                right_bottom_screen = (x, y)

        # # 외각선 그리기 
        # cv2.drawContours(image_with_contours, [largest_contour], 0, (0, 255, 255), 3)

        # 꼭지점 좌표
        src_points = np.float32([left_top_screen, right_top_screen, left_bottom_screen, right_bottom_screen])

        # 목표 평면 좌표
        width, height = 800, 600  # 목표 이미지 크기 
        dst_points = np.float32([[0, 0], [width, 0], [0, height], [width, height]])

        # 원근 변환 행렬 계산
        perspective_matrix = cv2.getPerspectiveTransform(src_points, dst_points)

        # 이미지를 평면화
        warped_image = cv2.warpPerspective(binary, perspective_matrix, (width, height))

        # BGR채널 순서를 RGB채널로 변경
        RGB_img = cv2.cvtColor(warped_image, cv2.COLOR_BGR2RGB)

        # R,G,B 채널 나누기
        R_img1, G_img1, B_img1 = cv2.split(RGB_img)

        R_plus = np.zeros((RGB_img.shape[0],RGB_img.shape[1]),dtype=np.ubyte)
        G_plus = np.zeros((RGB_img.shape[0],RGB_img.shape[1]),dtype=np.ubyte)
        B_plus = np.zeros((RGB_img.shape[0],RGB_img.shape[1]),dtype=np.ubyte)

        N = 150 # 해당 값을 기준으로 이진화

        # 영상 이진화 하기
        for h in range(RGB_img.shape[0]):
            for w in range(RGB_img.shape[1]):
                if(np.int32(B_img1[h,w])>N):
                    R_img1[h,w] = G_img1[h,w] = B_img1[h,w] = 255
                else:
                    R_img1[h,w] = G_img1[h,w] = B_img1[h,w] = 0

        RGB_img[:,:,0] = R_img1; RGB_img[:,:,1] = G_img1; RGB_img[:,:,2] = B_img1; 

        crop_image = img_crop(RGB_img, x_val, y_val, w_val, h_val)



        # 평면화 및 이진화된 이미지를 저장
        cv2.imwrite(f"{output_path}/{file_name}/{cnt}.jpg", crop_image)

        print(f"{output_path}/{file_name}/{cnt}.jpg")

../img_data/processed_data/1/1.jpg
../img_data/processed_data/1/2.jpg
../img_data/processed_data/1/3.jpg
../img_data/processed_data/1/4.jpg
../img_data/processed_data/1/5.jpg
../img_data/processed_data/1/6.jpg
../img_data/processed_data/1/7.jpg
../img_data/processed_data/1/8.jpg
../img_data/processed_data/1/9.jpg
../img_data/processed_data/1/10.jpg
../img_data/processed_data/1/11.jpg
../img_data/processed_data/1/12.jpg
../img_data/processed_data/1/13.jpg
../img_data/processed_data/1/14.jpg
../img_data/processed_data/1/15.jpg
../img_data/processed_data/1/16.jpg
../img_data/processed_data/1/17.jpg
../img_data/processed_data/1/18.jpg
../img_data/processed_data/1/19.jpg
../img_data/processed_data/1/20.jpg
../img_data/processed_data/1/21.jpg
../img_data/processed_data/1/22.jpg
../img_data/processed_data/1/23.jpg
../img_data/processed_data/1/24.jpg
../img_data/processed_data/1/25.jpg
../img_data/processed_data/1/26.jpg
../img_data/processed_data/1/27.jpg
../img_data/processed_data/1/28.jpg
.